In [58]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict
from scipy.stats import zscore
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import re
from rapidfuzz import process, fuzz
import pyodbc


In [79]:


# DATA________________________________
orders_df=pd.read_csv("Final.csv")
menu_df=pd.read_csv("menu.csv")
category_df=pd.read_csv("categories.csv")
df=pd.read_csv("Bill_detail.csv")

In [85]:
menu=menu_df["ItemName"].unique().tolist()

In [ ]:
df[df["Item"]]

In [87]:
df[(df["item_ID"]!=0)&(~df["ItemName"].isin(menu))]["ItemName"].value_counts()

ItemName
Delivery Charge                469
Combo                          293
Jnash                          204
Cheese HotDog Spring Rolls       2
Name: count, dtype: int64

In [73]:

category_dict=dict(zip(category_df["ItemName"],category_df["CatName"]))
orders_df["Category"]=orders_df["Item"].map(category_dict)

In [75]:
category_dict["Jnash "]

'Junior style'

In [76]:
orders_df.loc[orders_df["Item"]=="Jnash","Category"]="Junior style"

In [78]:
orders_df[orders_df["Item"]=="Jnash"]

,Bill_ID,item_ID,QUANTITY,PRICE,Total,ItemType,ParentID,item_notes,Item,Category,day,Date,Hour,period,date_only
2,143115,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Saturday,2025-04-26 21:16:05.387,21.0,evening,2025-04-26
9,143117,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Saturday,2025-04-26 23:42:49.100,23.0,evening,2025-04-26
40,144118,113710,1.0,190.0,230.0,1.0,0.0,combo,Jnash,Junior style,Monday,2025-04-28 18:45:19.403,18.0,evening,2025-04-28
59,144129,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Monday,2025-04-28 22:15:28.620,22.0,evening,2025-04-28
85,144139,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Tuesday,2025-04-29 20:28:33.170,20.0,evening,2025-04-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4864,167234,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Tuesday,2025-06-24 22:18:48.953,22.0,evening,2025-06-24
4891,167249,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Wednesday,2025-06-25 15:51:12.077,15.0,afternoon,2025-06-25
4901,167252,113710,2.0,190.0,380.0,1.0,0.0,(),Jnash,Junior style,Wednesday,2025-06-25 17:51:22.990,17.0,afternoon,2025-06-25
4922,167261,113710,1.0,190.0,190.0,1.0,0.0,(),Jnash,Junior style,Wednesday,2025-06-25 19:41:58.090,19.0,evening,2025-06-25


In [ ]:


items = menu_df["ItemName"].unique().tolist()
def best_match(item, candidates, threshold=80):
    best_score = 0
    best_candidate = None
    for cand in candidates:
        score = fuzz.token_sort_ratio(item, cand)
        if score > best_score and score >= threshold:
            best_score = score
            best_candidate = cand
    return best_candidate

def extract_notes(txt):
    irrelevant_patterns = [
        r'delivery order difference', r'maxi cola difference', r'regular & spicy',
        r'still have', r'with order', r'cup with order', r'with delivery'
    ]

    note_patterns = [
        r'without [\w\s]+', r'no [\w\s]+', r'same but with [\w\s]+',
        r'replace it with [\w\s]+', r'topped with [\w\s]+', r'top with [\w\s]+',
        r'with extra [\w\s]+', r'with [\w\s]+', r'extra [\w\s]+',
        r'add [\w\s]+', r'for my [\w\s]+', r'dont add [\w\s]+',
        r'do not add [\w\s]+', r'no extra [\w\s]+', r'without extra [\w\s]+',
        r'instead [\w\s]+', r'[\w\s]+ instead of [\w\s]+', r'only [\w\s]+',
        r'[\w\s]+ only', r'on top [\w\s]+', r'divide [\w\s]+', r'split [\w\s]+',
        r'full [\w\s]+', r'half [\w\s]+', r'non [\w\s]+', r'not [\w\s]+',
        r'make it [\w\s]+', r'don\'t [\w\s]+', r'you [\w\s]+', r'in [\w\s]+',
        r'^i [\w\s]+', r'combo [\w\s]+',
    ]

    note_txt = []
    for pattern in irrelevant_patterns:
        matches = re.findall(pattern, txt)
        for match in matches:
            note_txt.append(match.strip())
            txt = txt.replace(match, "")

    for pattern in note_patterns:
        matches = re.findall(pattern, txt)
        for match in matches:
            note_txt.append(match.strip())
            txt = txt.replace(match, "")

    cleaned = re.sub(r'\s+', ' ', txt).strip()
    return cleaned, note_txt

def clean(order):
    order = order.lower().strip()
    order = re.sub(r"[^a-z0-9\s]", '', order)  # remove special characters
    order = re.sub(r"\s+", ' ', order)  # collapse multiple spaces
    order = re.sub(r"\band\b", '', order)  # remove 'and'
    order = order.strip()
    order, _ = extract_notes(order)
    if order not in items:
        order = best_match(order, items, threshold=80)
    return order
def split_orders(df, item_col='Item'):
    rows = []

    for _, row in df.iterrows():
        raw_items = row[item_col].lower().replace(',', ' and ').split(' and ')
        for raw_item in raw_items:
            cleaned_item = clean(raw_item)
            if cleaned_item:
                
                new_row = row.copy()
                new_row['Item'] = cleaned_item
                rows.append(new_row)

    result_df = pd.DataFrame(rows)

    if item_col in result_df.columns:
        result_df.drop(columns=[item_col], inplace=True)

    return result_df

In [50]:

orders_df=split_orders(orders_df)
# Date________________________________
orders_df['Date'] = pd.to_datetime(orders_df['Date'])
orders_df['Hour'] = orders_df['Date'].dt.hour

# Build category mapping
item_categories = menu_df.set_index('item_ID')['Category'].to_dict()

# Menu fallback by category
menu_by_cat = menu_df.groupby('Category')['item_ID'].apply(list).to_dict()
menu_ids = menu_df['item_ID'].unique().tolist()

#Association Rules from orders ────────────────────────────────────────────
transactions = orders_df.groupby("Bill_ID")['item_ID'].apply(list).tolist()
te = TransactionEncoder().fit(transactions)
# df_trans = pd.DataFrame(te.transform(transactions), columns=te.columns_)
te_ary = te.transform(transactions)              # this is an ndarray, not sparse
df_trans = pd.DataFrame(te_ary, columns=te.columns_)

freq_itemsets = apriori(df_trans, min_support=0.002, use_colnames=True)
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)

rules = rules[
    (rules['confidence'] > 0.15) &
    (rules['lift'] > 1.05) &
    (rules['antecedents'].apply(len) <= 2)
].reset_index(drop=True)
rules['ant_str'] = rules['antecedents'].apply(lambda x: tuple(sorted(x)))
rules['con_str'] = rules['consequents'].apply(lambda x: tuple(sorted(x)))
rules = rules.drop_duplicates(subset=['ant_str','con_str']).drop(columns=['ant_str','con_str'])

# Co‑purchase vectors for “content” sim ────────────────────────────────────

item_list = df_trans.columns.tolist()     
item_to_index = {it: i for i, it in enumerate(item_list)}
item_vectors  = df_trans.T.values          

# Time‑of‑day z‑scores ──────────────────────────────────────────────────────

def get_period(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'
    
orders_df['period'] = orders_df['Hour'].apply(get_period)

periodly_counts = (
    orders_df.groupby(['period','item_ID'])
            .size()
            .unstack(fill_value=0)
)
periodly_z = periodly_counts.apply(zscore, axis=1).fillna(0)
# Candidate gatherer ───────────────────────────────────────────────────────
def gather_candidates(cart_items, rules, orders_df, menu_ids):
    cart_set = set(cart_items)
    cands = set()

    # a) from rules
    for _, r in rules.iterrows():
        if r['antecedents'].issubset(cart_set):
            cands |= set(r['consequents'])

    # b) time‑tops this hour
    current_period=get_period(datetime.now().hour)
    tops = (
        orders_df[orders_df['period']==current_period]['item_ID']
                .value_counts()
                .head(10)
                .index
                .tolist()
    )
    cands |= set(tops)

    # c) cold‑start: full menu
    cands |= set(menu_ids)

    return list(cands - cart_set)

# ── 5) Scoring function ─────────────────────────────────────────────────────────
def compute_scores(cart_items, candidates, rules,
                periodly_z, item_to_index, item_vectors, orders_df):
    cart_set = set(cart_items)
    idxs = [item_to_index[i] for i in cart_set if i in item_to_index]
    centroid = item_vectors[idxs].mean(axis=0, keepdims=True) if idxs else None

    sold_set = set(orders_df['item_ID'])
    current_period=get_period(datetime.now().hour)

    rows = []
    for it in candidates:
        # rule_score
        match = rules[rules['consequents'].apply(lambda cs: it in cs)]
        rule_score = (match['lift']*match['confidence']).max() if not match.empty else 0.0

        # time_pop
        time_pop = periodly_z.loc[current_period, it] if it in periodly_z.columns else 0.0

        # freshness boost
        freshness = 1.0 if it not in sold_set else 0.0

        # co‑purchase sim
        cont_sim = 0.0
        if centroid is not None and it in item_to_index:
            vec = item_vectors[item_to_index[it]].reshape(1,-1)
            cont_sim = cosine_similarity(centroid, vec)[0,0]

        rows.append({
            'item': it,
            'rule_score': rule_score,
            'time_pop': time_pop,
            'freshness': freshness,
            'content_sim': cont_sim
        })
    return pd.DataFrame(rows)

# ── 6) Unified recommend ────────────────────────────────────────────────────────
def unified_recommend(cart_items,
                    rules=rules, orders_df=orders_df, menu_ids=menu_ids,
                    item_to_index=item_to_index, item_vectors=item_vectors,
                    alpha=1.0, beta=0.5, gamma=0.2, sigma=0.3,
                    top_per_cat=2):
    cands     = gather_candidates(cart_items, rules, orders_df, menu_ids)
    df_scores = compute_scores(cart_items, cands, rules,
                            periodly_z, item_to_index, item_vectors, orders_df)

    df_scores['score'] = (
        alpha * df_scores['rule_score'] +
        beta  * df_scores['time_pop']    +
        gamma * df_scores['freshness']   +
        sigma * df_scores['content_sim']
    )
    df_scores['Category'] = df_scores['item'].map(item_categories)

    out = {}
    for cat in ['main','side','drink']:
        sub = df_scores[df_scores['Category']==cat]
        top = sub.nlargest(top_per_cat, 'score')['item'].tolist()

        # explicit fill from menu if short
        if len(top) < top_per_cat:
            for m in menu_by_cat.get(cat, []):
                if m not in cart_items and m not in top:
                    top.append(m)
                    if len(top)==top_per_cat:
                        break
        out[cat] = top

    return out

In [57]:
menu_df

,item_ID,ItemName,SellPrice,ItemType,Category
0,113708,JOG,160.0,1,main
1,113709,Jvolt,165.0,1,drink
2,113710,Jnash,190.0,1,main
3,113711,WildEpic,210.0,1,main
4,113712,WildBlaze,210.0,1,main
5,113713,WildNash,250.0,1,main
6,113714,4PcOG,220.0,1,main
7,113715,4PcVolt,220.0,1,drink
8,113716,4PcNash,280.0,1,main
9,113717,8PcOG,440.0,1,main


In [52]:
cart     = [113741,113711]
# cart=[]
recs     = unified_recommend(
    cart_items    = cart,
    rules         = rules,
    orders_df     = orders_df,
    menu_ids      = menu_ids,
    item_to_index = item_to_index,
    item_vectors  = item_vectors,
    top_per_cat=2
)


In [53]:
menu_dict=dict(zip(menu_df["item_ID"],menu_df["ItemName"]))
for cat in (["main","side","drink"]):
    print(cat,": ")

    for item in recs[cat]:
        print(menu_dict.get(item),end=" ")
    print()

main : 
Jnash Cheesy Waffle'd 
side : 
Cheese Spring Rolls French Fries 
drink : 
v7 cola Jvolt 


# Recommendation Engine Class

In [ ]:
class RecommendationEngine:
    DEFAULT_ALPHA = 1.0
    DEFAULT_BETA  = 0.7
    DEFAULT_GAMMA = 0.5
    DEFAULT_SIGMA = 0.3
    def __init__(
        self,
        orders_df: pd.DataFrame,
        menu_df: pd.DataFrame,
        rule_support: float = 0.002,
        rule_confidence: float = 0.15,
        rule_lift: float = 1.05
    ):
        # Initialize data
        self.orders_df = orders_df.copy()
        self.menu_df = menu_df.copy()
        self.alpha = RecommendationEngine.DEFAULT_ALPHA
        self.beta  = RecommendationEngine.DEFAULT_BETA
        self.gamma = RecommendationEngine.DEFAULT_GAMMA
        self.sigma = RecommendationEngine.DEFAULT_SIGMA

        # Preprocess date & period
        self.orders_df['Date'] = pd.to_datetime(self.orders_df['Date'])
        self.orders_df['Hour'] = self.orders_df['Date'].dt.hour
        self.orders_df['period'] = self.orders_df['Hour'].apply(self._get_period)
        # Category maps
        self.item_categories = self.menu_df.set_index('item_ID')['Category'].to_dict()
        self.menu_by_cat = self.menu_df.groupby('Category')['item_ID'].apply(list).to_dict()
        self.menu_ids = self.menu_df['item_ID'].unique().tolist()
        # Association rules
        self._build_association_rules(rule_support, rule_confidence, rule_lift)
        # Co-purchase vectors
        self._build_item_vectors()
        # Period-based z-scores
        self._build_periodly_z()

    def _get_period(self, hour: int) -> str:
        if 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'afternoon'
        elif 18 <= hour < 24:
            return 'evening'
        return 'night'

    def _build_association_rules(self, support, min_conf, min_lift):
        tx = self.orders_df.groupby('Bill_ID')['item_ID'].apply(list).tolist()
        te = TransactionEncoder().fit(tx)
        df_trans = pd.DataFrame(te.transform(tx), columns=te.columns_)
        freq_itemsets = apriori(df_trans, min_support=support, use_colnames=True)
        rules = association_rules(freq_itemsets, metric='lift', min_threshold=1)
        # filter
        rules = rules[
            (rules['confidence'] > min_conf) &
            (rules['lift'] > min_lift) &
            (rules['antecedents'].apply(len) <= 2)
        ].reset_index(drop=True)
        # dedupe
        rules['ant_str'] = rules['antecedents'].apply(lambda x: tuple(sorted(x)))
        rules['con_str'] = rules['consequents'].apply(lambda x: tuple(sorted(x)))
        rules = rules.drop_duplicates(subset=['ant_str','con_str'])
        self.rules = rules.drop(columns=['ant_str','con_str'])
        # preserve df_trans for vectors
        self._df_trans = df_trans

    def _build_item_vectors(self):
        df_trans = self._df_trans
        self.item_list = df_trans.columns.tolist()
        self.item_to_index = {it: i for i, it in enumerate(self.item_list)}
        self.item_vectors = df_trans.T.values

    def _build_periodly_z(self):
        counts = (
            self.orders_df.groupby(['period','item_ID'])
                        .size()
                        .unstack(fill_value=0)
        )
        self.periodly_z = counts.apply(zscore, axis=1).fillna(0)

    def gather_candidates(self, cart_items: list[int]) -> list[int]:
        cart_set = set(cart_items)
        cands = set()
        # rules
        for _, r in self.rules.iterrows():
            if r['antecedents'].issubset(cart_set):
                cands |= set(r['consequents'])
        # period-tops
        current_period = self._get_period(datetime.now().hour)
        tops = (
            self.orders_df[self.orders_df['period'] == current_period]['item_ID']
            .value_counts().head(10).index.tolist()
        )
        cands |= set(tops)
        # cold start
        cands |= set(self.menu_ids)
        return list(cands - cart_set)

    def compute_scores(
        self, cart_items: list[int], candidates: list[int]
    ) -> pd.DataFrame:
        cart_set = set(cart_items)
        idxs = [self.item_to_index[i] for i in cart_set if i in self.item_to_index]
        centroid = self.item_vectors[idxs].mean(axis=0, keepdims=True) if idxs else None
        sold_set = set(self.orders_df['item_ID'])
        current_period = self._get_period(datetime.now().hour)
        rows = []
        for it in candidates:
            match = self.rules[self.rules['consequents'].apply(lambda cs: it in cs)]
            rule_score = (match['lift']*match['confidence']).max() if not match.empty else 0.0
            time_pop = self.periodly_z.loc[current_period, it] if it in self.periodly_z.columns else 0.0
            freshness = 1.0 if it not in sold_set else 0.0
            cont_sim = 0.0
            if centroid is not None and it in self.item_to_index:
                vec = self.item_vectors[self.item_to_index[it]].reshape(1, -1)
                cont_sim = cosine_similarity(centroid, vec)[0,0]
            rows.append({
                'item': it,
                'rule_score': rule_score,
                'time_pop': time_pop,
                'freshness': freshness,
                'content_sim': cont_sim
            })
        return pd.DataFrame(rows)

    def recommend(
        self, cart_items: list[int],
        top_per_cat: int = 3
    ) -> dict[int, list[int]]:
        cands = self.gather_candidates(cart_items)
        df_scores = self.compute_scores(cart_items, cands)
        df_scores['score'] = (
            self.alpha * df_scores['rule_score'] +
            self.beta  * df_scores['time_pop'] +
            self.gamma * df_scores['freshness'] +
            self.sigma * df_scores['content_sim']
        )
        df_scores['Category'] = df_scores['item'].map(self.item_categories)
        out = {}
        for cat in ['main','side','drink']:
            subset = df_scores[df_scores['Category']==cat]
            top = subset.nlargest(top_per_cat, 'score')['item'].tolist()
            if len(top) < top_per_cat:
                for m in self.menu_by_cat.get(cat, []):
                    if m not in cart_items and m not in top:
                        top.append(m)
                        if len(top) == top_per_cat:
                            break
            out[cat] = top
        return out


In [55]:

# Example usage:
engine = RecommendationEngine(orders_df, menu_df)
recs = engine.recommend([113708,113709], top_per_cat=2)
print(recs)


{'main': [113710, 113711], 'side': [113753, 113742], 'drink': [113757, 113715]}


In [56]:
for cat in (["main","side","drink"]):
    print(cat,": ")

    for item in recs[cat]:
        print(menu_dict.get(item),end=" ")
    print()

main : 
Jnash WildEpic 
side : 
Cheese Spring Rolls French Fries 
drink : 
v7 cola 4PcVolt 
